<a href="https://colab.research.google.com/github/das-nishant/ML-Projects-notebook/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 : Fake News  


0 : real news

importing the Dependencies

In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score




In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
#printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [7]:
news_dataset = pd.read_csv('/content/FakeNewsNet.csv')

Data Pre-Processing

In [ ]:
news_dataset.shape

(23196, 5)

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [8]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [ ]:
 # replacing the null value with empty string
news_dataset = news_dataset.fillna('')


In [ ]:
# mergin the news_url and news title
news_dataset['content'] = news_dataset['news_url']+' '+news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        http://toofab.com/2017/05/08/real-housewives-a...
1        https://www.today.com/style/see-people-s-choic...
2        https://www.etonline.com/news/220806_sophia_bu...
3        https://www.dailymail.co.uk/news/article-33655...
4        https://www.zerchoo.com/entertainment/gossip-g...
                               ...                        
23191    https://www.express.co.uk/news/royal/807049/pi...
23192    hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...
23193    http://www.justjared.com/2018/01/17/jessica-ch...
23194    www.intouchweekly.com/posts/tristan-thompson-f...
23195    https://www.billboard.com/articles/news/bbma/8...
Name: content, Length: 23196, dtype: object


In [ ]:
print(news_dataset.columns)


Index(['title', 'news_url', 'source_domain', 'tweet_num', 'real', 'content'], dtype='object')


In [ ]:
# separating the data and label
x = news_dataset.drop(columns='real', axis=1)
y = news_dataset['real']

In [ ]:
print(x)
print (y)

                                                   title  ...                                            content
0      Kandi Burruss Explodes Over Rape Accusation on...  ...  http://toofab.com/2017/05/08/real-housewives-a...
1      People's Choice Awards 2018: The best red carp...  ...  https://www.today.com/style/see-people-s-choic...
2      Sophia Bush Sends Sweet Birthday Message to 'O...  ...  https://www.etonline.com/news/220806_sophia_bu...
3      Colombian singer Maluma sparks rumours of inap...  ...  https://www.dailymail.co.uk/news/article-33655...
4      Gossip Girl 10 Years Later: How Upper East Sid...  ...  https://www.zerchoo.com/entertainment/gossip-g...
...                                                  ...  ...                                                ...
23191  Pippa Middleton wedding: In case you missed it...  ...  https://www.express.co.uk/news/royal/807049/pi...
23192  Zayn Malik & Gigi Hadid’s Shocking Split: Why ...  ...  hollywoodlife.com/2018/03/13/zayn

stemming


In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        http toofab com real housew atlanta kandi burr...
1        http www today com style see peopl choic award...
2        http www etonlin com news sophia bush send swe...
3        http www dailymail co uk news articl colombian...
4        http www zerchoo com entertain gossip girl yea...
                               ...                        
23191    http www express co uk news royal pippa middle...
23192    hollywoodlif com zayn malik gigi hadid split g...
23193    http www justjar com jessica chastain recal mo...
23194    www intouchweekli com post tristan thompson fe...
23195    http www billboard com articl news bbma kelli ...
Name: content, Length: 23196, dtype: object


In [ ]:
# separating the data and real
x = news_dataset['content'].values
y = news_dataset['real'].values

In [ ]:
print(x)

['http toofab com real housew atlanta kandi burruss rape phaedra park porsha william kandi burruss explod rape accus real housew atlanta reunion video'
 'http www today com style see peopl choic award red carpet look peopl choic award best red carpet look'
 'http www etonlin com news sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva'
 ...
 'http www justjar com jessica chastain recal moment mother boyfriend slap kick genit jessica chastain recal moment mother boyfriend slap kick genit'
 'www intouchweekli com post tristan thompson feel dump khloe kardashian tristan thompson feel dump khlo kardashian refus let move la home exclus'
 'http www billboard com articl news bbma kelli clarkson medley bbma kelli clarkson perform medley kendrick lamar humbl hit billboard music award']


In [ ]:
print(y)

[1 1 1 ... 1 0 1]


In [ ]:
y.shape

(23196,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer = vectorizer.fit(x)
x = vectorizer.transform(x)

In [ ]:
print(x)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 316924 stored elements and shape (23196, 17314)>
  Coords	Values
  (0, 175)	0.14434726829473674
  (0, 896)	0.3389455172701989
  (0, 2218)	0.4346705393896895
  (0, 3144)	0.028363875555072016
  (0, 5230)	0.19914481139678233
  (0, 7204)	0.27967207824608226
  (0, 7240)	0.03029315079920102
  (0, 8136)	0.4346705393896895
  (0, 11301)	0.16947275863509945
  (0, 11540)	0.2113369959982094
  (0, 11846)	0.21414819505876584
  (0, 12356)	0.33456306448701023
  (0, 12418)	0.2501629274621897
  (0, 12736)	0.1394415242605404
  (0, 15510)	0.1652746695506423
  (0, 16345)	0.1037611672890413
  (0, 16828)	0.12458641416073078
  (1, 985)	0.33310329340670436
  (1, 1532)	0.18067383197494744
  (1, 2474)	0.4135533362491038
  (1, 2851)	0.4642049540481571
  (1, 3144)	0.04487540286284784
  (1, 7240)	0.04792777148735079
  (1, 9039)	0.36163482558348325
  (1, 11465)	0.25473967851260193
  :	:
  (23194, 8547)	0.17169083170481492
  (23194, 8810)	0.211463587763149

splitting the dataset to training and test data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=2)


Training the model ; logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(x_train, y_train)

LogisticRegression()

Evalution

accuracy score


In [ ]:
# accuracy score onm the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [ ]:
print('Accuracy score of the training data :' , training_data_accuracy)

Accuracy score of the training data : 0.9483724940719983


In [ ]:
# accuracy score on the test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [ ]:
print('Accuracy score of the test data :' , test_data_accuracy)

Accuracy score of the test data : 0.9269396551724138


Making a Predictive system

In [ ]:
x_new = x_test[0]

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [ ]:
print(y_test[3])

0
